In [7]:
import pandas as pd 
import numpy as np 
import os 
import pathlib 
from tqdm import tqdm 

import re

from helper_functions import * 
path_to_all_fa = "/Volumes/HNSD01/storage/ref/hg19"
path_to_sample_readdf = "/Volumes/HNSD01/storage/TSMA_validation_readdf/reads/1-MYAAAA29TS_M525-M725"
outputdir = "./output_methyl_ReadBased_Models"

samplename = path_to_sample_readdf.split("/")[-1]
path_to_save_output = f"./{outputdir}/{samplename}"
print(f"Working on sample {samplename}, output will be saved to {path_to_save_output}")

os.makedirs(path_to_save_output, exist_ok=True)

all_panels = ["Rare_cancers_panel",
              "TMD_450_panel",
              "TSMA_panel"]

panel = "TSMA_panel"
all_files = [item for item in pathlib.Path(path_to_sample_readdf).glob(f"{panel}/*.csv")]

input_file = all_files[0]
tmpdf = pd.read_csv(input_file, index_col = [0])
region = input_file.name.replace("readdf_", "").replace(".csv", "")

Working on sample 1-MYAAAA29TS_M525-M725, output will be saved to ././output_methyl_ReadBased_Models/1-MYAAAA29TS_M525-M725


In [ ]:
region_chrom = f"chr{region.split("_")[0]}"
region_start = int(region.split("_")[1])
region_end = int(region.split("_")[2])

refseq_at_cluster = get_refseq(path_to_all_fa = path_to_all_fa, 
                                chrom = region_chrom, 
                                start = region_start, 
                                end = region_end + 1)
all_cpg_in_cluster = [m.start(0) for m in re.finditer("CG", refseq_at_cluster)]
cpg_coords = [item + region_start for item in all_cpg_in_cluster]

df = tmpdf.copy()

pattern = re.compile("^[1-9][0-9]M")
df["check_cigar"] = df["cigar"].apply(lambda x: bool(pattern.fullmatch(x)))
df = df[df["check_cigar"] == True]
            
df["end"] = df[["start", "cigar"]].apply(lambda x: int(x[0]) + int(x[1].replace("M", "")), axis = 1)

for cpg_pos in cpg_coords:
    df[cpg_pos] = df[["start", "end", "seq"]].apply(lambda x: get_CpG_status(x[0], x[1], x[2], cpg_pos, mode = "num"), axis = 1)
    
betadf = pd.DataFrame(data = df["sample"].unique())
betadf.columns = ["sample"]
for cpg_pos in tqdm(cpg_coords):    
    tmpdf = df[["sample", cpg_pos]].copy()
    tmpcountdf = tmpdf.groupby('sample')[cpg_pos].apply(lambda x: (x == 1).sum()/((x == 0).sum() + (x == 1).sum()) ).reset_index(name= "meth_level_{}".format(cpg_pos))
    betadf = betadf.merge(tmpcountdf[["sample", "meth_level_{}".format(cpg_pos)]], right_on = "sample", left_on = "sample", how = "outer")
        

FastaNotFoundError: Cannot read FASTA from file /Volumes/HNSD01/storage/ref/hg19/11.fa